In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from IPython.display import Image
from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import seaborn as sns; sns.set()
from keras.preprocessing.text import Tokenizer
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from keras.utils import to_categorical
import nltk
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
%matplotlib inline 

Using TensorFlow backend.


In [2]:
#Set data to only use the 6 categories chosen
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med', 'rec.sport.hockey', 'rec.motorcycles', 'sci.space']

twenty_train = fetch_20newsgroups(subset = 'train', categories=categories, shuffle=True, random_state=42)
twenty_train.target_names


['alt.atheism',
 'comp.graphics',
 'rec.motorcycles',
 'rec.sport.hockey',
 'sci.med',
 'sci.space',
 'soc.religion.christian']

In [3]:
len(twenty_train.data)
len(twenty_train.filenames)

4048

In [4]:
print("\n".join(twenty_train.data[1].split("\n")[:20]))


From: sandvik@newton.apple.com (Kent Sandvik)
Subject: Re: Genocide is Caused by Atheism
Organization: Cookamunga Tourist Bureau
Lines: 26

In article <1993Apr19.113255.27550@monu6.cc.monash.edu.au>,
darice@yoyo.cc.monash.edu.au (Fred Rice) wrote:
> >Fred, the problem with such reasoning is that for us non-believers
> >we need a better measurement tool to state that person A is a
> >real Muslim/Christian, while person B is not. As I know there are
> >no such tools, and anyone could believe in a religion, misuse its
> >power and otherwise make bad PR. It clearly shows the sore points
> >with religion -- in other words show me a movement that can't spin
> >off Khomeinis, Stalins, Davidians, Husseins... *).
> 
> I don't think such a system exists.  I think the reason for that is an
> condition known as "free will".  We humans have got it.  Anybody, using
> their free-will, can tell lies and half-truths about *any* system and
> thus abuse it for their own ends.



In [5]:
twenty_train.target[:10]


array([6, 0, 4, 3, 1, 0, 2, 4, 1, 2], dtype=int64)

In [6]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names)

['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'rec.sport.hockey', 'sci.med', 'sci.space', 'soc.religion.christian']
['alt.atheism', 'comp.gr

In [7]:
#Use count vectorizer method to transform data to vectors
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(4048, 51970)

In [8]:
#Get words most frequently used
from sklearn.feature_extraction.text import TfidfTransformer

transform = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = transform.transform(X_train_counts)
X_train_tf.shape

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape



(4048, 51970)

In [9]:

#Test classifier to see if it can correctly predic where something should go 
classifier = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

test = ['In space no one will hear you scream', 'Computer needs more RAM']
counts = count_vect.transform(test)
tfidf = tfidf_transformer.transform(counts)

prediction = classifier.predict(tfidf)

for doc, category in zip(test, prediction):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'In space no one will hear you scream' => sci.space
'Computer needs more RAM' => comp.graphics


In [10]:
#Use pipeline method to create the classifier work flow
textClassifier = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])


textClassifier.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [11]:
#test and evaluate
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = textClassifier.predict(docs_test)
np.mean(predicted == twenty_test.target)       

0.8730040846639435

In [12]:
#For SVM the pipeline just needs to be adjusted
textClassifier = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

textClassifier.fit(twenty_train.data, twenty_train.target)  

predicted = textClassifier.predict(docs_test)
np.mean(predicted == twenty_test.target)        

0.9279613813590791

In [13]:
#Use metrics method to get report on how classifier did
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))
                                    
confusion = metrics.confusion_matrix(twenty_test.target, predicted)
pd.DataFrame(data = confusion, columns = twenty_train.target_names)

                        precision    recall  f1-score   support

           alt.atheism       0.94      0.77      0.85       319
         comp.graphics       0.90      0.95      0.93       389
       rec.motorcycles       0.96      0.99      0.98       398
      rec.sport.hockey       0.97      1.00      0.98       399
               sci.med       0.95      0.85      0.90       396
             sci.space       0.94      0.94      0.94       394
soc.religion.christian       0.85      0.95      0.90       398

              accuracy                           0.93      2693
             macro avg       0.93      0.92      0.92      2693
          weighted avg       0.93      0.93      0.93      2693



,alt.atheism,comp.graphics,rec.motorcycles,rec.sport.hockey,sci.med,sci.space,soc.religion.christian
0,246,4,3,3,10,4,49
1,3,371,1,1,1,8,4
2,0,1,396,1,0,0,0
3,0,1,0,398,0,0,0
4,6,19,10,7,336,6,12
5,0,13,1,1,5,372,2
6,7,3,1,1,1,5,380


In [14]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[246,   4,   3,   3,  10,   4,  49],
       [  3, 371,   1,   1,   1,   8,   4],
       [  0,   1, 396,   1,   0,   0,   0],
       [  0,   1,   0, 398,   0,   0,   0],
       [  6,  19,  10,   7, 336,   6,  12],
       [  0,  13,   1,   1,   5, 372,   2],
       [  7,   3,   1,   1,   1,   5, 380]], dtype=int64)

In [15]:
train = fetch_20newsgroups()
test = fetch_20newsgroups(subset='test')

print("Train")
print(train.data[0].strip())
print("\nLABEL:", train.target_names[train.target[0]],
      "=", train.target[0])
print("Test")
print(test.data[-1].strip())
print("\nLABEL:", test.target_names[test.target[-1]],
      "=", test.target[-1])

Train
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----

LABEL: rec.autos = 7
Test
From: adamsj@gtewd.mtv.gtegsc.com
Subject: Re: Homosexuality issues in Christianity
Reply-To: adamsj@gtewd.mtv.gtegsc.com
Organization: GTE Govt. Systems, Electronics Def. Div.
Lines: 18

In article <May.13.02.29.39.1993.1505@geneva.rutgers.edu>, rev

In [16]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def tokenizer(text):
       return text.split()

def tokenizer_stemmer(text):
    return [stemmer.stem(word) for word in tokenizer(text)]#text.split()]


def stop_removal(text):
       return [w for w in text if not w in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lewis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
#Use method so network knows that abbreviations such as don't should be do not

import re

vocabulary = (
    (re.compile(r"\bdon't\b"), "do not"),
    (re.compile(r"\bit's\b"), "it is"),
    (re.compile(r"\bi'm\b"), "i am"),
    (re.compile(r"\bi've\b"), "i have"),
    (re.compile(r"\bcan't\b"), "cannot"),
    (re.compile(r"\bdoesn't\b"), "does not"),
    (re.compile(r"\bthat's\b"), "that is"),
    (re.compile(r"\bdidn't\b"), "did not"),
    (re.compile(r"\bi'd\b"), "i would"),
    (re.compile(r"\byou're\b"), "you are"),
    (re.compile(r"\bisn't\b"), "is not"),
    (re.compile(r"\bi'll\b"), "i will"),
    (re.compile(r"\bthere's\b"), "there is"),
    (re.compile(r"\bwon't\b"), "will not"),
    (re.compile(r"\bwoudn't\b"), "would not"),
    (re.compile(r"\bhe's\b"), "he is"),
    (re.compile(r"\bthey're\b"), "they are"),
    (re.compile(r"\bwe're\b"), "we are"),
    (re.compile(r"\blet's\b"), "let us"),
    (re.compile(r"\bhaven't\b"), "have not"),
    (re.compile(r"\bwhat's\b"), "what is"),
    (re.compile(r"\baren't\b"), "are not"),
    (re.compile(r"\bwasn't\b"), "was not"),
    (re.compile(r"\bwouldn't\b"), "would not"),
)

def fix_apostrophes(text):
    text = text.lower()
    
    for pattern, replacement in vocabulary:
        text = pattern.sub(replacement, text)

    return text

text_train = list(map(fix_apostrophes, train.data))
text_test = list(map(fix_apostrophes, test.data))

In [18]:
#Limit what is used to what was found in training set
Unique_Words= 10000

tokenizer = Tokenizer(
    num_words=Unique_Words,
    lower=False,
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

In [19]:

tokenizer.fit_on_texts(text_train)
print('Found %s unique tokens.' % len(tokenizer.word_index))

# the 10000 most frequent training words are used

# generate index vectors from both train and test

seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)

Found 126595 unique tokens.


In [20]:

#Use pad sequences method so only equally sized vectors are used 
Seq_length= 1000

x_train = pad_sequences(seq_train, maxlen=Seq_length, truncating='post')
x_test = pad_sequences(seq_test, maxlen=Seq_length, truncating='post')

y_train = to_categorical(np.asarray(train.target))
y_test = to_categorical(np.asarray(test.target))
print('Training set size:', len(train.data))
print('Training data shape:', x_train.shape)
print('Training label shape:', y_train.shape)
print('\nTest set size:', len(test.data))
print('Test data shape:', x_test.shape)
print('Test label shape:', y_test.shape)

Training set size: 11314
Training data shape: (11314, 1000)
Training label shape: (11314, 20)

Test set size: 7532
Test data shape: (7532, 1000)
Test label shape: (7532, 20)


In [21]:
#Build network

embedding_size=32
model = Sequential()
model.add(Embedding(input_dim=10000,
 output_dim=embedding_size,
 input_length=1000))
model.add(SimpleRNN(units=embedding_size))

model.add(Dense(20, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy',
 metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                660       
Total params: 322,740
Trainable params: 322,740
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Train network
model.fit(x_train, y_train, epochs=10, batch_size=60, validation_split=0.2)



C:\Users\Lewis\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9051 samples, validate on 2263 samples
Epoch 1/10
9051/9051 [==============================] - 107s 12ms/step - loss: 0.2765 - acc: 0.9244 - val_loss: 0.1983 - val_acc: 0.9500
Epoch 2/10
9051/9051 [==============================] - 112s 12ms/step - loss: 0.1968 - acc: 0.9500 - val_loss: 0.1958 - val_acc: 0.9500
Epoch 3/10
9051/9051 [==============================] - 160s 18ms/step - loss: 0.1902 - acc: 0.9500 - val_loss: 0.1888 - val_acc: 0.9500
Epoch 4/10
9051/9051 [==============================] - 157s 17ms/step - loss: 0.1777 - acc: 0.9504 - val_loss: 0.1811 - val_acc: 0.9511
Epoch 5/10
9051/9051 [==============================] - 120s 13ms/step - loss: 0.1633 - acc: 0.9526 - val_loss: 0.1762 - val_acc: 0.9530
Epoch 6/10
9051/9051 [==============================] - 105s 12ms/step - loss: 0.1490 - acc: 0.9545 - val_loss: 0.1742 - val_acc: 0.9549
Epoch 7/10
9051/9051 [==============================] - 106s 12ms/step - loss: 0.1354 - acc: 0.9568 - val_loss: 0.1726 - val_acc: 

In [23]:
acc = model.evaluate(x_test, y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

7532/7532 [==============================] - 32s 4ms/step
Test loss is 0.19 accuracy is 0.95 
